# 🦜 LangChain + Cynosure Bridge Integration

Полное руководство по использованию LangChain с Cynosure Bridge для доступа к Claude через OpenAI API.

## 📦 Установка зависимостей

In [ ]:
!pip install langchain langchain-openai langchain-community chromadb faiss-cpu tiktoken

## 🚀 Базовая настройка LangChain

In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Настройка для Cynosure Bridge
os.environ["OPENAI_API_KEY"] = "dummy-key"  # Любой ключ
os.environ["OPENAI_API_BASE"] = "http://192.168.1.196:3000/v1"

# Создание LLM через Cynosure Bridge
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.1,
    base_url="http://192.168.1.196:3000/v1",
    api_key="dummy-key"
)

# Создание Embeddings модели
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    base_url="http://192.168.1.196:3000/v1",
    api_key="dummy-key"
)

print("✅ LangChain настроен для работы с Cynosure Bridge")

## 💬 Простые цепочки (Chains)

In [ ]:
# Простая цепочка с промптом
prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты - полезный AI ассистент. Отвечай кратко и по делу."),
    ("user", "{question}")
])

# Создание цепочки
chain = prompt | llm | StrOutputParser()

# Тестирование
response = chain.invoke({"question": "Что такое LangChain?"})
print(f"🤖 Ответ: {response}")

## 🔄 Streaming цепочки

In [ ]:
# Streaming LLM
streaming_llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.3,
    streaming=True,
    base_url="http://192.168.1.196:3000/v1",
    api_key="dummy-key"
)

streaming_chain = prompt | streaming_llm | StrOutputParser()

# Демонстрация streaming
print("🌊 Streaming ответ:")
for chunk in streaming_chain.stream({"question": "Расскажи интересную историю про AI"}):
    print(chunk, end="", flush=True)
print("\n✅ Streaming завершен")

## 📚 Работа с документами и векторными хранилищами

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Подготовка документов
documents = [
    Document(
        page_content="Cynosure Bridge - это OpenAI-совместимый прокси для Claude MAX. Он позволяет использовать Claude через стандартный OpenAI API без необходимости платить за API ключи.",
        metadata={"source": "cynosure_docs", "topic": "introduction"}
    ),
    Document(
        page_content="LangChain - это фреймворк для разработки приложений с использованием языковых моделей. Он предоставляет инструменты для создания сложных AI приложений.",
        metadata={"source": "langchain_docs", "topic": "framework"}
    ),
    Document(
        page_content="Векторные базы данных позволяют выполнять семантический поиск по документам. FAISS - одна из самых популярных библиотек для этого.",
        metadata={"source": "vector_docs", "topic": "search"}
    ),
    Document(
        page_content="RAG (Retrieval Augmented Generation) - это техника, которая комбинирует поиск релевантной информации с генерацией ответов языковой моделью.",
        metadata={"source": "rag_docs", "topic": "technique"}
    )
]

# Разбиение текста на чанки
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)
splits = text_splitter.split_documents(documents)

# Создание векторного хранилища
vectorstore = FAISS.from_documents(splits, embeddings)

print(f"📚 Создано векторное хранилище с {len(splits)} документами")

## 🔍 Retrieval и поиск

In [ ]:
# Создание retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)

# Тестирование поиска
query = "Что такое RAG?"
relevant_docs = retriever.invoke(query)

print(f"🔍 Поиск по запросу: '{query}'")
print(f"📄 Найдено {len(relevant_docs)} релевантных документов:")

for i, doc in enumerate(relevant_docs, 1):
    print(f"\n{i}. Topic: {doc.metadata['topic']}")
    print(f"   Content: {doc.page_content[:100]}...")

## 🤖 RAG Chain с LangChain

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

# RAG промпт
rag_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""Используй следующий контекст для ответа на вопрос. 
Если информации недостаточно, честно об этом скажи.

Контекст:
{context}

Вопрос: {question}

Ответ:"""
)

def format_docs(docs):
    """Форматирует документы в строку"""
    return "\n\n".join([doc.page_content for doc in docs])

# Создание RAG цепочки
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# Тестирование RAG
questions = [
    "Что такое Cynosure Bridge?",
    "Объясни RAG технику", 
    "Как работает LangChain?",
    "Что такое семантический поиск?"
]

for question in questions:
    print(f"\n❓ Вопрос: {question}")
    answer = rag_chain.invoke(question)
    print(f"🤖 Ответ: {answer}")
    print("-" * 50)

## 💾 Персистентное хранилище с ChromaDB

In [ ]:
from langchain_community.vectorstores import Chroma
import tempfile
import os

# Создание временной директории для ChromaDB
persist_directory = tempfile.mkdtemp()

# Создание Chroma векторного хранилища
chroma_db = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory
)

print(f"💾 ChromaDB создан в: {persist_directory}")

# Тестирование поиска в ChromaDB
chroma_retriever = chroma_db.as_retriever(search_kwargs={"k": 2})
chroma_results = chroma_retriever.invoke("векторный поиск")

print(f"\n🔍 ChromaDB поиск по 'векторный поиск':")
for doc in chroma_results:
    print(f"📄 {doc.metadata['topic']}: {doc.page_content[:80]}...")

## 🧠 Memory и Conversation

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Создание хранилища истории
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Conversational chain с памятью
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты - полезный ассистент. Помни предыдущие сообщения в разговоре."),
    ("placeholder", "{history}"),
    ("user", "{input}")
])

conversational_chain = conversational_prompt | llm | StrOutputParser()

# Добавление памяти к цепочке
with_message_history = RunnableWithMessageHistory(
    conversational_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# Тестирование разговора с памятью
session_id = "test_session_1"

response1 = with_message_history.invoke(
    {"input": "Привет! Меня зовут Алексей."},
    config={"configurable": {"session_id": session_id}}
)
print(f"🤖 Ответ 1: {response1}")

response2 = with_message_history.invoke(
    {"input": "Как меня зовут?"},
    config={"configurable": {"session_id": session_id}}
)
print(f"🤖 Ответ 2: {response2}")

response3 = with_message_history.invoke(
    {"input": "Расскажи про LangChain"},
    config={"configurable": {"session_id": session_id}}
)
print(f"🤖 Ответ 3: {response3}")

## 🔧 Agents и Tools

In [ ]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.tools import tool
from langchain import hub
import requests
import json

# Создание custom tools
@tool
def get_cynosure_health() -> str:
    """Проверяет состояние Cynosure Bridge сервера."""
    try:
        response = requests.get("http://192.168.1.196:3000/health", timeout=5)
        return json.dumps(response.json(), indent=2)
    except Exception as e:
        return f"Ошибка при проверке: {str(e)}"

@tool
def search_documents(query: str) -> str:
    """Ищет информацию в базе знаний по запросу."""
    try:
        docs = retriever.invoke(query)
        if not docs:
            return "Релевантные документы не найдены."
        
        results = []
        for doc in docs:
            results.append(f"Topic: {doc.metadata['topic']} - {doc.page_content}")
        
        return "\n\n".join(results)
    except Exception as e:
        return f"Ошибка поиска: {str(e)}"

# Список инструментов
tools = [get_cynosure_health, search_documents]

# Получение промпта для агента
try:
    prompt = hub.pull("hwchase17/openai-functions-agent")
except:
    # Fallback промпт если hub недоступен
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Ты полезный ассистент с доступом к инструментам."),
        ("user", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ])

# Создание агента
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("🤖 Agent создан с инструментами:")
for tool in tools:
    print(f"  🔧 {tool.name}: {tool.description}")

In [ ]:
# Тестирование агента
test_queries = [
    "Проверь состояние Cynosure Bridge",
    "Найди информацию про RAG",
    "Что ты знаешь о LangChain?"
]

for query in test_queries:
    print(f"\n❓ Запрос: {query}")
    print("=" * 50)
    result = agent_executor.invoke({"input": query})
    print(f"\n✅ Результат: {result['output']}")
    print("-" * 50)

## 📊 LangSmith Tracing (опционально)

In [ ]:
# Настройка LangSmith для трейсинга (если доступен)
import os

# Раскомментируйте если у вас есть LangSmith API ключ
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "your-langsmith-api-key"
# os.environ["LANGCHAIN_PROJECT"] = "cynosure-bridge-demo"

# Пример трейсинга цепочки
with_tracing = chain
traced_response = with_tracing.invoke({"question": "Объясни преимущества Cynosure Bridge"})
print(f"📊 Traced response: {traced_response}")
print("💡 Трейсы можно посмотреть в LangSmith dashboard")

## 🧪 Продвинутые техники

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Structured output с Pydantic
class TechAnalysis(BaseModel):
    technology: str = Field(description="Название технологии")
    pros: List[str] = Field(description="Преимущества")
    cons: List[str] = Field(description="Недостатки")
    use_cases: List[str] = Field(description="Случаи использования")
    rating: int = Field(description="Рейтинг от 1 до 10")

# Создание парсера
parser = PydanticOutputParser(pydantic_object=TechAnalysis)

# Промпт для структурированного вывода
analysis_prompt = PromptTemplate(
    template="""Проанализируй следующую технологию и предоставь структурированный анализ.

{format_instructions}

Технология: {technology}

Анализ:""",
    input_variables=["technology"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Цепочка с парсингом
analysis_chain = analysis_prompt | llm | parser

# Тестирование структурированного вывода
tech_analysis = analysis_chain.invoke({"technology": "LangChain"})

print("🔍 Структурированный анализ LangChain:")
print(f"📊 Технология: {tech_analysis.technology}")
print(f"✅ Преимущества: {', '.join(tech_analysis.pros)}")
print(f"❌ Недостатки: {', '.join(tech_analysis.cons)}")
print(f"🎯 Применение: {', '.join(tech_analysis.use_cases)}")
print(f"⭐ Рейтинг: {tech_analysis.rating}/10")

## 🎯 Заключение

Этот notebook демонстрирует полную интеграцию LangChain с Cynosure Bridge:

- ✅ **Базовые цепочки** с промптами
- ✅ **Streaming** в реальном времени
- ✅ **Векторные хранилища** (FAISS, ChromaDB)
- ✅ **RAG реализация** с retrieval
- ✅ **Memory и conversation** история
- ✅ **Agents и tools** для сложных задач
- ✅ **Structured output** с Pydantic

### 🚀 Преимущества комбинации:

1. **Бесплатный доступ к Claude** через MAX подписку
2. **Экосистема LangChain** со всеми инструментами
3. **OpenAI API совместимость** для легкой миграции
4. **Локальное развертывание** без зависимости от внешних API